In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
from shapely.geometry import shape, Point, Polygon

# Crime

In [2]:
crime = pd.read_csv('crime.csv')
crime.head(10)

,results_key,id,case_number,date,year,primary_type,description,domestic,arrest,beat,...,district,latitude,longitude,location_description,fbi_code,iucr,entered_on,x_coordinate,y_coordinate,updated
0,1,11561837,JC110056,2018-12-31 23:59:00,2018,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,0,0,734,...,7,41.763180,-87.657707,NaN,11,1153,2019-01-17 14:26:36,NaN,NaN,2019-04-25 03:10:46
1,2,11556487,JC104662,2018-12-31 23:59:00,2018,CRIMINAL DAMAGE,TO VEHICLE,0,0,2211,...,22,41.689079,-87.696068,STREET,14,1320,2019-01-10 15:16:50,NaN,NaN,2019-04-25 03:10:46
2,3,11552699,JC100043,2018-12-31 23:57:00,2018,CRIMINAL DAMAGE,TO PROPERTY,0,0,613,...,6,41.740520,-87.647392,APARTMENT,14,1310,2019-01-10 15:16:50,NaN,NaN,2019-04-25 03:10:46
3,4,11552724,JC100006,2018-12-31 23:56:00,2018,BATTERY,AGG: HANDS/FIST/FEET NO/MINOR INJURY,0,1,1233,...,12,41.857067,-87.657623,OTHER,08B,0440,2019-01-10 15:16:50,NaN,NaN,2019-04-25 03:10:46
4,5,11552731,JC100031,2018-12-31 23:55:00,2018,BATTERY,DOMESTIC BATTERY SIMPLE,0,0,621,...,6,41.751915,-87.647720,APARTMENT,08B,0486,2019-01-10 15:16:50,NaN,NaN,2019-04-25 03:10:46
5,6,11552715,JC100026,2018-12-31 23:49:00,2018,BATTERY,AGGRAVATED: HANDGUN,0,0,1522,...,15,41.875683,-87.760483,STREET,04B,041A,2019-03-20 16:10:43,NaN,NaN,2019-04-25 03:10:46
6,7,11552741,JC100011,2018-12-31 23:48:00,2018,BATTERY,DOMESTIC BATTERY SIMPLE,1,0,612,...,6,41.750153,-87.661011,APARTMENT,08B,0486,2019-01-10 15:16:50,NaN,NaN,2019-04-25 03:10:46
7,8,11552602,JC100089,2018-12-31 23:47:00,2018,BATTERY,SIMPLE,0,0,1931,...,19,41.939625,-87.673996,"VEHICLE - OTHER RIDE SHARE SERVICE (E.G., UBER...",08B,0460,2019-01-10 15:16:50,NaN,NaN,2019-04-25 03:10:46
8,9,11554852,JC101652,2018-12-31 23:45:00,2018,CRIMINAL DAMAGE,TO PROPERTY,0,0,1422,...,14,41.905563,-87.707588,APARTMENT,14,1310,2019-01-10 15:16:50,NaN,NaN,2019-04-25 03:10:46
9,10,11553488,JC101094,2018-12-31 23:45:00,2018,THEFT,FROM BUILDING,0,0,1924,...,19,41.940517,-87.654121,BAR OR TAVERN,06,0890,2019-01-10 15:16:50,NaN,NaN,2019-04-25 03:10:46


In [3]:
crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257121 entries, 0 to 257120
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   results_key           257121 non-null  int64  
 1   id                    257121 non-null  int64  
 2   case_number           257121 non-null  object 
 3   date                  257121 non-null  object 
 4   year                  257121 non-null  int64  
 5   primary_type          257121 non-null  object 
 6   description           257121 non-null  object 
 7   domestic              257121 non-null  int64  
 8   arrest                257121 non-null  int64  
 9   beat                  257121 non-null  int64  
 10  block                 257121 non-null  object 
 11  community_area        257119 non-null  float64
 12  ward                  257118 non-null  float64
 13  district              257121 non-null  int64  
 14  latitude              257121 non-null  float64
 15  

In [4]:
crime.drop(columns=['x_coordinate', 'y_coordinate'], inplace=True)

In [5]:
crime.primary_type.value_counts()

THEFT                                62582
BATTERY                              48605
CRIMINAL DAMAGE                      27139
ASSAULT                              19915
DECEPTIVE PRACTICE                   17369
OTHER OFFENSE                        16542
NARCOTICS                            12497
BURGLARY                             11399
MOTOR VEHICLE THEFT                   9561
ROBBERY                               9349
CRIMINAL TRESPASS                     6687
WEAPONS VIOLATION                     5321
OFFENSE INVOLVING CHILDREN            2034
CRIM SEXUAL ASSAULT                   1554
PUBLIC PEACE VIOLATION                1338
INTERFERENCE WITH PUBLIC OFFICER      1280
SEX OFFENSE                           1025
PROSTITUTION                           705
HOMICIDE                               569
ARSON                                  368
LIQUOR LAW VIOLATION                   258
GAMBLING                               197
STALKING                               194
KIDNAPPING 

In [6]:
def point_lookup(polygon_dict, point):
    
    for i in range(len(list(polygon_dict.keys()))):
        result = Polygon(polygon_dict[list(polygon_dict.keys())[i]]).contains(point) # zero index is there because of zoning type in zoning data
        
        if result == True:
            return list(polygon_dict.keys())[i]
            break 

In [7]:
positions = pd.DataFrame(zip(crime.longitude, crime.latitude))

In [9]:
with open('neighorhood_polys.json','r') as f:
    neighborhoods = json.load(f)

In [10]:
crime['true_neighborhood'] = np.nan
for i in range(len(crime)):
    crime['true_neighborhood'][i] = point_lookup(neighborhoods, Point(positions.iloc[i,:].values))

/Users/mark/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mark/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [22]:
crime_by_neighborhood_by_category = pd.DataFrame(crime.groupby(['true_neighborhood', 'primary_type'])['primary_type'].count())

In [25]:
crime_by_neighborhood_by_category[crime_by_neighborhood_by_category.primary_type.isna()]

,,primary_type
true_neighborhood,primary_type,


In [26]:
crime_by_neighborhood_by_category

primary_type
true_neighborhood primary_type                        
Albany Park       ARSON                              1
                  ASSAULT                          150
                  BATTERY                          460
                  BURGLARY                         145
                  CRIM SEXUAL ASSAULT               12
...                                                ...
Wrigleyville      OTHER OFFENSE                     19
                  PUBLIC PEACE VIOLATION             4
                  ROBBERY                           20
                  SEX OFFENSE                        2
                  THEFT                            139

[2207 rows x 1 columns]